# Introduction
Welcome to the first practical for Graph Representation Learning. In this practical, we will be covering the content of the lectures about TransE.

We will be using [PyTorch](https://pytorch.org/docs/stable/index.html) to implement TransE from scratch, building it up piece by piece.

The main goal of this practical is to create a working implementation of TransE. There are also two optional parts: *filtered negative sampling* and *RotatE*.

The notebook is divided into sections, each of which comes with complete or partially completed code. Before each snippet of code there will be a description of what we are about to implement. The sections of code you need to complete are marked as **Tasks**. The majority of the length of this practical comes from code already written for you, so don't panic at the apparent length! There are only 8 tasks for you to complete.

Please ensure that you operate within the framework given in the notebook and bring any questions you may have to the practical demonstrators. We suggest that you **DO NOT** edit code that is a part of the framework, since this will make it more difficult for demonstrators to assist if your code is broken.

Since we are working in a Jupyter Notebook, the code is very interactive. When you're stuck on something, try adding a new block of code below what you're working on and using it to debug your code. If you are new to Jupyter Notebooks, see [here](https://www.youtube.com/watch?v=inN8seMm7UI&ab_channel=TensorFlow) for a brief introduction video. If you are using Google Colab (which we recommend doing), please ensure you have changed the runtime type to use a GPU, as it will make your code run much faster.

# Imports

Run the following blocks of code to install and import and the necessary python packages.

In [1]:
# This code cell is commented out as it is not working properly locally for me.
# %%capture
# !pip install pykeen

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pykeen
import argparse
import json
import os
import random

from torch.utils.data import Dataset, DataLoader
from torch.utils import data as torch_data
from sklearn.metrics import average_precision_score
from pykeen.datasets import Nations
from typing import List, Tuple

C:\Users\jupit\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(torch.cuda.is_available())

True


# Dataset

## Loading Nations from `pykeen`
We will use `pykeen` to load the `Nations` dataset, which is a small knowledge graph with 14 entities, 55 relations, and 1992 triples describing countries and their political relationships.

We first define a function to convert the `pykeen` datasets to lists of triples. We then create 3 lists of triples: one for training, one for validation, and one for testing.

In [4]:
def create_triples_from_pykeen_dataset(dataset: pykeen.triples.triples_factory.TriplesFactory):
    slcwa_instances=dataset.create_slcwa_instances(
        batch_size=1,
        shuffle=True,
    )
    positive_dataset = [tuple(batch.positives[0].tolist()) for batch in slcwa_instances]
    return positive_dataset

In [5]:
dataset = Nations()
train_triples = create_triples_from_pykeen_dataset(dataset.training)
valid_triples = create_triples_from_pykeen_dataset(dataset.validation)
test_triples = create_triples_from_pykeen_dataset(dataset.testing)

In [6]:
# My code cell
print(type(train_triples))
print(train_triples[0])
print(len(train_triples)) # 1592 + 199 + 201 = 1992
print(len(valid_triples))
print(len(test_triples))

<class 'list'>
(12, 45, 5)
1592
199
201


#### Task 1
Define a function `id_triple_as_labels` that takes in a triple of entity/relation ids and returns a triple of labels.

*Hint: use the dictionaries `id2entity` and `id2relation` provided below*

In [7]:
id2entity = dataset.training.entity_id_to_label
id2relation = dataset.training.relation_id_to_label

### BEGIN SOLUTION
def id_triple_as_labels(triple: Tuple):
    head, relation, tail = triple
    return id2entity[head], id2relation[relation], id2entity[tail]
### END SOLUTION

We can then run the following cell to see some of the facts from the **Nations** dataset.

In [8]:
for i in range(10):
    print(train_triples[i])
    print(id_triple_as_labels(train_triples[i]))
# Note that, for instnace, we can know that 'uk' has id 11.

(12, 45, 5)
('usa', 'timesinceally', 'india')
(11, 17, 12)
('uk', 'exportbooks', 'usa')
(4, 19, 8)
('egypt', 'independence', 'jordan')
(5, 45, 12)
('india', 'timesinceally', 'usa')
(1, 14, 11)
('burma', 'embassy', 'uk')
(11, 7, 5)
('uk', 'commonbloc1', 'india')
(1, 8, 6)
('burma', 'commonbloc2', 'indonesia')
(12, 17, 0)
('usa', 'exportbooks', 'brazil')
(0, 20, 5)
('brazil', 'intergovorgs', 'india')
(13, 28, 9)
('ussr', 'ngoorgs3', 'netherlands')


## Negative Sampling
Simply training on positive facts will not suffice, since the model will then learn to just maximise the similarity measure for every possible fact in the database. Thus, for each positive fact, we need to sample a set of corrupted facts.

#### Task 2
First, define a function `get_corrupted_entities` that takes in a single positive sample and returns an array of corrupted entities. The positive sample is a tuple of integers, representing the IDs of the entities/relation.

The function should also take in `train_dataset`, as you will need to access `train_dataset.negative_sample_size` (the number of corrupted entities to sample), `train_dataset.nentity` (the number of entities in the knowledge graph), and possibly `train_dataset.true_head` / `train_dataset.true_tail` (dictionaries providing the set of all known true triples for the given head / tail). The keys for the train_dataset.true_head dictionary are tuples of the form (relation, tail), and the keys for train_dataset.true_tail are tuples of the form (head, relation). The full definition of `TrainDataset` can be found further below if you need to refer to it.

The function should also take in `mode`, which, if set to `mode == 'head'`, indicates that the head entity should be corrupted, and likewise the tail entity if `mode == 'tail'`. Your output should be a numpy array with shape `[train_dataset.negative_sample_size]`.

Note: as an optional extra, you can sample corrupted entities such that the resulting triples are not known to be true in the knowledge graph. If you get stuck on this, first implement a simpler solution first and then come back to it later.

In [9]:
# My code cell
# Check dictionary get() method
dict_1 = {
    'car' : 14,
    'love' : 21,
    'student' : 7
}
print(dict_1.get('csr'))
print(type(dict_1.get('csr')))
print(dict_1.get('csr',[]))
print(type(dict_1.get('csr',[])))

None
<class 'NoneType'>
[]
<class 'list'>


In [10]:
# My code cell
# Check list and set
set_1 = {1,2,2,3}
print(type(set_1))
set_2 = set([1,2,2,3])
print(set_2)
# set_3 = set(dict_1.get('csr')) # reason we need to use set(true_head.get((relation, tail), []))
# print(set_3)
set_4 = {}
print(type(set_4))

<class 'set'>
{1, 2, 3}
<class 'dict'>


In [10]:
# My code cell
# Check random.choices() method
list_1 = [1, 2, 3, 4, 5, 6, 7, 8, 9]
random.choices(list_1, k = 3) # allow repeated sampling
# May cause problem

[8, 6, 8]

In [12]:
# BEGIN SOLUTION
def get_corrupted_entities(positive_sample: Tuple, train_dataset: TrainDataset, mode: str) -> np.ndarray:
    head, relation, tail = positive_sample
    entities = list(range(train_dataset.nentity))  # Entities should be indexed from 0 to nentity - 1
    negative_sample = []

    # Access the true_head and true_tail dictionaries
    true_head, true_tail = train_dataset.true_head, train_dataset.true_tail

    if mode == 'head':
        # Get true head entities for this (relation, tail) pair, or an empty set if not in true_head
        true_head_entities = set(true_head.get((relation, tail), [])) # true_head.get((relation, tail)) is a numpy array
        # Filter entities not in true head set for efficient sampling
        possible_corruptions = [e for e in entities if e not in true_head_entities]
        negative_sample = random.choices(possible_corruptions, k=train_dataset.negative_sample_size)

    elif mode == "tail":
        # Get true tail entities for this (head, relation) pair, or an empty set if not in true_tail
        true_tail_entities = set(true_tail.get((head, relation), []))
        # Filter entities not in true tail set for efficient sampling
        possible_corruptions = [e for e in entities if e not in true_tail_entities]
        negative_sample = random.choices(possible_corruptions, k=train_dataset.negative_sample_size)

    return np.array(negative_sample)
### END SOLUTION

#### Task 3
Next, we can create the negative samples using the corrupted entities. Define a function `get_negative_sample` which takes in a positive sample and corrupted head/tail entities, which will be created by the `get_corrupted_entities` function we defined above (do not call to the above function, the corrupted entities will be passed in as arguments).

The argument `positive_sample` is a tuple `(head, relation, tail)`. The arguments `corrupted_head_entities` and `corrupted_tail_entities` should have shape `[negative_sample_size]` each.

Your function should return a `numpy` array with shape `[2*negative_sample_size, 3]`, by combining the corrupted entities with the positive sample.

In [13]:
### BEGIN SOLUTION
# corrupted_head_entities is the output of get_corrupted_entities fucntion above
def get_negative_sample(positive_sample, corrupted_head_entities, corrupted_tail_entities):
    corrupted_head_samples = np.zeros((corrupted_head_entities.shape[0], 3))
    corrupted_head_samples[:, 0] = corrupted_head_entities
    corrupted_head_samples[:, 1:3] = positive_sample[1:3]

    corrupted_tail_samples = np.zeros((corrupted_tail_entities.shape[0], 3))
    corrupted_tail_samples[:, 2] = corrupted_tail_entities
    corrupted_tail_samples[:, 0:2] = positive_sample[0:2]

    return np.concatenate((corrupted_head_samples, corrupted_tail_samples), axis=0)
### END SOLUTION

## Train Dataset
Now that we have written the functions we need to perform our negative sampling, let's combine everything together to create our `torch` training dataset.

Notice that in the `__get_item__` function, we convert the samples to `torch` tensors before we return them. Up to this point, we have been working with `numpy` arrays; `torch` tensors have the same structure, but are optimised to run on the `GPU` and can also track gradients to be used for optimising parameters.

In [14]:
class TrainDataset(Dataset):
    def __init__(self, triples, nentity, nrelation, negative_sample_size):
        self.len = len(triples)
        self.triples = triples # all training triples
        self.triple_set = set(triples) # unique triples
        self.nentity = nentity # number of entities in the knowledge graph
        self.nrelation = nrelation # number of relations in the knowledge graph
        self.negative_sample_size = negative_sample_size // 2 # Half from heads, half from tails

        # known triples for the given heads / tails
        self.true_head, self.true_tail = self.get_true_head_and_tail(self.triples)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        '''
        Get an item from the Dataset.
        '''
        # Fetch a positive sample
        positive_sample = self.triples[idx]

        # Sample corrupted head and tail entities
        corrupted_head_entities = get_corrupted_entities(positive_sample, self, 'head')
        corrupted_tail_entities = get_corrupted_entities(positive_sample, self, 'tail')

        # Create the negative sample
        negative_sample = get_negative_sample(positive_sample, corrupted_head_entities, corrupted_tail_entities)

        # Convert samples to torch tensors
        negative_sample = torch.LongTensor(negative_sample)
        positive_sample = torch.LongTensor(positive_sample)

        return positive_sample, negative_sample

    @staticmethod
    def collate_fn(data):
        positive_sample = torch.stack([_[0] for _ in data], dim=0)
        negative_sample = torch.stack([_[1] for _ in data], dim=0)
        return positive_sample, negative_sample

    @staticmethod
    def get_true_head_and_tail(triples):
        '''
        Build a dictionary of true triples that will
        be used to filter these true triples for negative sampling
        '''

        true_head = {}
        true_tail = {}

        for head, relation, tail in triples:
            if (head, relation) not in true_tail:
                true_tail[(head, relation)] = []
            true_tail[(head, relation)].append(tail)
            if (relation, tail) not in true_head:
                true_head[(relation, tail)] = []
            true_head[(relation, tail)].append(head)

        for relation, tail in true_head:
            true_head[(relation, tail)] = np.array(list(set(true_head[(relation, tail)])))
        for head, relation in true_tail:
            true_tail[(head, relation)] = np.array(list(set(true_tail[(head, relation)])))

        return true_head, true_tail


## Test Dataset
We will use a seperate dataset for our testing, with the batch size always set to 1. The test dataset will have two modes: `'head-batch'` and `'tail-batch'`. In the first mode, the dataset should return a positive sample and a list of all possible heads, and similarly for the second mode.

Since we are doing filtered evaluation, we do not want triples which are known to be true to affect the ranking. Thus, we will filter the heads / tails out of our samples that yield triples which are known to be true.

#### Task 4
Write a function `get_filtered_test_sample` that takes in a positive triple (`head, relation, tail`) and the `test_dataset` (which can be found further below). The function should return a list of size `test_dataset.nentity`, where each element is the ID of an entity. It is important for the downstream ranking that we ensure the list is this size.

To filter the sample, if for some entity `x`, `(x, relation, tail)` already appears in the set of known triples, instead replace it with `head`. The set of known triples can be accessed by `test_dataset.triple_set`. You will need to check the value of `test_dataset.mode` and return a list of entities accordingly.

In [15]:
# BEGIN SOLUTION
def get_filtered_test_sample(head, relation, tail, test_dataset: Dataset) -> List:
    filtered_sample = list(range(test_dataset.nentity))
    
    if test_dataset.mode == 'head-batch':
        # Replace entities that form known triples in 'head-batch' mode
        for i in range(test_dataset.nentity):
            if (i, relation, tail) in test_dataset.triple_set:
                filtered_sample[i] = head  # Replace with head if the triple is known

    elif test_dataset.mode == 'tail-batch':
        # Replace entities that form known triples in 'tail-batch' mode
        for i in range(test_dataset.nentity):
            if (head, relation, i) in test_dataset.triple_set:
                filtered_sample[i] = tail  # Replace with tail if the triple is known

    else:
        raise ValueError(f'Negative batch mode {test_dataset.mode} not supported')
    return filtered_sample
### END SOLUTION

Now we can use our filtered test sampling function to define our test dataset.

In [16]:
class TestDataset(Dataset):
    def __init__(self, triples, all_true_triples, nentity, nrelation, mode):
        self.len = len(triples)
        self.triple_set = set(all_true_triples) # set of all known true triples
        self.triples = triples # test triples
        self.nentity = nentity
        self.nrelation = nrelation
        self.mode = mode # 'head-batch' or 'tail-batch'

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        head, relation, tail = self.triples[idx] # fetch a positive sample from the test triples

        # get the filtered sample using the function we defined
        filtered_sample = get_filtered_test_sample(head, relation, tail, self)

        # convert to torch tensors
        filtered_sample = torch.LongTensor(filtered_sample)
        positive_sample = torch.LongTensor((head, relation, tail))

        return positive_sample, filtered_sample, self.mode

    @staticmethod
    def collate_fn(data):
        positive_sample = torch.stack([_[0] for _ in data], dim=0)
        negative_sample = torch.stack([_[1] for _ in data], dim=0)
        mode = data[0][2]
        return positive_sample, negative_sample, mode

## Dataset Iterator
As a final step towards constructing our datasets, we define a class that allows us to convert `torch` dataloaders into python iterators, which will make it simpler for us to define training and testing step functions.

In [17]:
class OneShotIterator:
    def __init__(self, dataloader):
        self.iterator = self.one_shot_iterator(dataloader)

    def __next__(self):
        return next(self.iterator)

    @staticmethod
    def one_shot_iterator(dataloader):
        '''
        Transform a PyTorch Dataloader into python iterator
        '''
        while True:
            for data in dataloader:
                yield data

We will create the actual train and test datasets in code further below, but here follows some code for constructing them, in case you would like to use them for debugging.

In [18]:
# make the train dataloader
train_dataloader = DataLoader(
    TrainDataset(train_triples,
                 14, # nentity
                 55, # nrelation
                 128), # negative sampling size
    batch_size=500,
    shuffle=True,
    # num_workers=1, original code
    num_workers=0,
    collate_fn=TrainDataset.collate_fn
)
# convert to an iterator
train_iterator = OneShotIterator(train_dataloader)
# get a sample from the train iterator
positive_sample, negative_sample = next(train_iterator)

If your code is correct, the following output should be `torch.Size([500, 128, 3])`.

In [19]:
negative_sample.size()

torch.Size([500, 128, 3])

In [20]:
# make the test dataloader
known_true_triples = train_triples + valid_triples + test_triples
test_dataloader_head = DataLoader(
    TestDataset(
        test_triples,
        known_true_triples,
        14, # nentity
        55, # nrelation
        'head-batch'
    ),
    batch_size=1,
    # num_workers=1,
    num_workers=0,
    collate_fn=TestDataset.collate_fn
)

If your code is correct, the following output should be `torch.Size([1, 14])`.

In [21]:
for positive_sample, negative_sample, mode in test_dataloader_head:
    print(negative_sample.size())
    break

torch.Size([1, 14])


# Model
We now define our model, `KGEModel`. It is built in such a way that we can implement different dissimilarity measures within it.

From here onwards, we will be working with `torch` tensors instead of `numpy` arrays, so make sure you are using `torch` operations.

## Parameter Initialisation
We will use `torch.nn.Parameter` to store our embeddings for entities and relations. We define a function `init_params` which initialises an embedding tensor of the given size and randomly samples values from the uniform distribution `[-embedding_range, embedding_range]`.

In [22]:
def init_params(tensor_size: tuple, embedding_range: float) -> nn.Parameter:
    embedding = nn.Parameter(torch.zeros(tensor_size))
    nn.init.uniform_(
        tensor=embedding,
        a=-embedding_range,
        b=embedding_range
    )
    return embedding

## Scoring Function
Different KG embedding models use different dissimilarity measures (aka scoring functions). We will define one for **TransE** and optionally define one for **RotatE**.

#### Task 5
Define a scoring function for **TransE** that takes in the head, relation, and tail, and returns a score for the triple. Each argument tensor has size `[batch_size, embedding_size]`. You may use either the $l_1$ or $l_2$ norm. Your output tensor should have size `[batch_size]`.

In [23]:
### BEGIN SOLUTION
# head, relation, tail are of shape [batch_size, embedding_size]
def TransE(head : torch.tensor, relation : torch.tensor, tail : torch.tensor, norm_type=1):
    translation = head + relation - tail
    if norm_type == 1:
        score = torch.norm(translation, p=1, dim=1)  # l1 norm
    elif norm_type == 2:
        score = torch.norm(translation, p=2, dim=1)  # l2 norm
    else:
        raise ValueError("norm_type must be either 1 (l1) or 2 (l2)")
    # score is of shape [batch_size]
    # the higher the score is, the poor the performance of the model and rank increases
    return score
### END SOLUTION

### Task 6 (Optional)
This is an optional task. You should get **TransE** working completely first and then come back to this.

Define a scoring function for **RotatE**. The head and tail will have size `[batch_size, 2 * embedding_size]` to store both the real and imaginary parts of the entities. *Hint: you can use `torch.chunk()` to split the tensor into its real and imaginary components*.

The relation will have size `[batch_size, embedding_size]`, representing the phase $\theta$ of the relation. *Hint: The real and imaginary components of the relation can be computed with `torch.cos` and `torch.sin`*.

#### Using Euler's Formula for Rotation
Let us write down the formula for rotating the entities explicitly.
We know **head entity** $\mathbf{h}$ has the following complex embedding:
$
\mathbf{h}=\mathbf{h}_{\text {real }}+i \cdot \mathbf{h}_{\text {imag }}
$
and each element $r_i$ of the relation embedding $\mathbf{r}$ is of the form $\theta_{r,i}$ (but in the slides, the entries are given by $e^{i \theta_{r,i}}$ with $|r_i| = 1$).

According to Euler's formula, a complex number $z=$ $a+i \cdot b$ can be rotated by an angle $\theta$ by multiplying it by

$$
\begin{aligned}
& e^{i \theta}=\cos (\theta)+i \sin (\theta) \\
& \quad z^{\prime}=z \cdot e^{i \theta}=(a+i \cdot b)(\cos (\theta)+i \cdot \sin (\theta))
\end{aligned}
$$


When we apply this rotation to each dimension $i$ of $h$ by $\theta_{r, i}$, we get:

$$
\begin{aligned}
& h_{\mathrm{real}, i}^{\prime}=h_{\mathrm{real}, i} \cdot \cos \left(\theta_{r, i}\right)-h_{\mathrm{imag}, i} \cdot \sin \left(\theta_{r, i}\right) \\
& h_{\mathrm{imag}, i}^{\prime}=h_{\mathrm{real}, i} \cdot \sin \left(\theta_{r, i}\right)+h_{\mathrm{imag}, i} \cdot \cos \left(\theta_{r, i}\right)
\end{aligned}
$$


This produces the rotated real and imaginary components of $h$ for each dimension.

In [24]:
# My code cell for checking `torch.chunk()` function
tensor_2 = torch.arange(4)
print(tensor_2)
print(tensor_2.size())
print(type(tensor_2.chunk(2)))
tensor_2.chunk(2)[0]
tensor_3 = torch.randn(2,4)
print(tensor_3)
tensor_3.chunk(2, dim=0)


tensor([0, 1, 2, 3])
torch.Size([4])
<class 'tuple'>
tensor([[ 0.3792, -0.8419, -0.7589, -0.1385],
        [ 0.5661, -0.4204, -0.6449,  0.3077]])


(tensor([[ 0.3792, -0.8419, -0.7589, -0.1385]]),
 tensor([[ 0.5661, -0.4204, -0.6449,  0.3077]]))

In [25]:
### BEGIN SOLUTION
def RotatE(head, relation, tail):
    # Split the head and tail into real and imaginary parts along the embedding size (last) dimension
    head_real, head_imag = torch.chunk(head, 2, dim=-1)
    tail_real, tail_imag = torch.chunk(tail, 2, dim=-1)

    # Compute real and imaginary parts of the rotation
    relation_real = torch.cos(relation) # of shape [batch_size]
    relation_imag = torch.sin(relation)

    # Rotate head
    rotated_head_real = head_real * relation_real - head_imag * relation_imag
    rotated_head_imag = head_real * relation_imag + head_imag * relation_real
    
    # Calculate the difference (dissimilarity) between rotated head and tail
    score_real = rotated_head_real - tail_real  # of shape [batch_size]
    score_imag = rotated_head_imag - tail_imag
    
    # Combine real and imaginary parts to get final score
    score = torch.norm(torch.cat([score_real, score_imag], dim=-1), p=2, dim=-1)

    return score
### END SOLUTION

## Full Model Definition
Now we can use our scoring function to define the model. Notice that in the `forward` function, we use `torch.index_select` to fetch the entity / relation embeddings from the their indices. `sample` is a tensor with the size `[batch_size, 3]`.

In [26]:
class KGEModel(nn.Module):
    def __init__(self, model_name: str, nentity: int, nrelation: int, hidden_dim: int, gamma: float,
                 double_entity_embedding: bool=False, double_relation_embedding: bool=False):
        super(KGEModel, self).__init__()
        self.model_name = model_name
        self.nentity = nentity
        self.nrelation = nrelation
        self.hidden_dim = hidden_dim
        self.epsilon = 2.0

        self.gamma = nn.Parameter(
            torch.Tensor([gamma]),
            requires_grad=False
        )

        self.embedding_range = nn.Parameter(
            torch.Tensor([(self.gamma.item() + self.epsilon) / hidden_dim]),
            requires_grad=False
        )

        self.entity_dim = 2*hidden_dim if double_entity_embedding else hidden_dim
        self.relation_dim = 2*hidden_dim if double_relation_embedding else hidden_dim

        # Create entity and relation embeddings
        self.entity_embedding = init_params(tensor_size=(nentity, self.entity_dim),
                                             embedding_range=self.embedding_range.item())
        self.relation_embedding = init_params(tensor_size=(nrelation, self.relation_dim),
                                              embedding_range=self.embedding_range.item())

        # This code supports easily adding new models like RotatE and ComplEx
        # Do not forget to modify this line when you add a new model in the "forward" function
        if model_name not in ['TransE', 'RotatE']:
            raise ValueError('model %s not supported' % model_name)

        if model_name == 'RotatE' and not double_entity_embedding:
            raise ValueError('RotatE should use --double_entity_embedding')
        if model_name == 'TransE' and double_entity_embedding:
            raise ValueError('TransE should not use --double_entity_embedding')

    def forward(self, sample):
        '''
        Forward function that calculate the score of a batch of triples.
        Sample is a batch of triples.
        '''
        head = torch.index_select(
            self.entity_embedding,
            dim=0,
            index=sample[:,0]
        )

        relation = torch.index_select(
            self.relation_embedding,
            dim=0,
            index=sample[:,1]
        )

        tail = torch.index_select(
            self.entity_embedding,
            dim=0,
            index=sample[:,2]
        )

        # Other models can be added here
        dissimilarity_measure = {
            'TransE': TransE,
            'RotatE': RotatE,
        }

        if self.model_name in dissimilarity_measure:
            score = dissimilarity_measure[self.model_name](head, relation, tail)
        else:
            raise ValueError('model %s not supported' % self.model_name)

        return score

# Training
In this section, we will first write a function to compute the model loss given a set of positive and negative samples, and then use it to define a single training step for the model.

#### Task 7
Before we can define a training step for our model, we must define a loss function for the model. We use negative sampling loss (from the **RotatE** paper), but without the self-adversarial parameter. Remember to refer to the lecture slides if you get stuck on this.

Define a function `get_model_loss` which takes in the `KGEModel`, the positive sample, and the negative sample, and returns a tuple of the loss, the positive sample loss, and the negative sample loss.

The positive sample will have size `[batch_size, 3]`, the negative sample will have size `[batch_size * negative_sampling_size, 3]`, and the margin $\gamma$ can be accessed through `model.gamma`.

In [27]:
# My code block for testing tensor size
tensor_1 = torch.zeros([2, 4], dtype=torch.int32)
print(tensor_1)
tensor_1.size()

tensor([[0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=torch.int32)


torch.Size([2, 4])

Recall that the negative sampling loss without the self-adversarial parameter is given by the following:
$$
\mathscr{L}=-\log \sigma(\gamma-d(\mathbf{h} \odot \mathbf{r}, \mathbf{t}))-\sum_{r\left(h^{\prime}, t^{\prime}) \in \mu[r(h, t)]\right.} \frac{1}{k} \log \sigma\left(d\left(\mathbf{h}^{\prime} \odot \mathbf{r}, \mathbf{t}^{\prime}\right)-\gamma\right),
$$
where $\gamma$ is a margin, $\sigma$ is the sigmoid function, and $\mu[r(h, t)]$ is a set of $k$ negative samples for $r(h, t)$.

In [28]:
### BEGIN SOLUTION
# calculate the time needed
from time import time

def get_model_loss(model: KGEModel, positive_sample: torch.tensor, negative_sample: torch.tensor) -> Tuple:

    '''
    for every positive_sample, we have negative_sampling_size many negative_sample
    positive_sample is of shape [batch_size, 3]
    negative_sample is of shape [batch_size * negative_sampling_size, 3]
    we use model.forward to compute the score
    '''

    loss = 0
    sample_loss = 0
    positive_sample_loss, negative_sample_loss = 0, 0
    batch_size = positive_sample.size(0)

    # Get the positive_sample_loss
    # - log(sigmoid(y-d(h,t)))
    positive_score = model(positive_sample) # of shape [batch_size]
    positive_sample_loss = -torch.log(torch.sigmoid(model.gamma - positive_score)) # of shape [batch_size]

    # Get the negative_sample_loss
    negative_score = model(negative_sample) # of shape [batch_size * negative_sampling_size]
    negative_sample_loss = -torch.log(torch.sigmoid(negative_score - model.gamma)) # of shape [batch_size * negative_sampling_size]
    negative_sample_loss = negative_sample_loss.view(batch_size, -1)
    negative_sample_loss = negative_sample_loss.mean(dim=1) # Now of shape [batch_size]


    sample_loss = positive_sample_loss + negative_sample_loss
    loss = sample_loss.mean()

    return loss, positive_sample_loss.mean(), negative_sample_loss.mean()


### END SOLUTION

We can now define a single train step for the model.

In [29]:
def train_step(model, optimizer, train_iterator, args):
    '''
    A single train step. Apply back-propation and return the loss
    '''

    model.train() # tell the torch model it's about to be trained

    optimizer.zero_grad() # explicitly set gradients to 0 before starting backprop

    positive_sample, negative_sample = next(train_iterator) # fetch samples from the dataset

    # reshape the negative sample
    # it will now have shape [batch_size * negative_sampling_size, 3]
    negative_sample = torch.reshape(negative_sample, (-1, 3))

    # move tensors to GPU
    if args.cuda:
        positive_sample = positive_sample.cuda()
        negative_sample = negative_sample.cuda()

    # compute the loss
    loss, positive_sample_loss, negative_sample_loss = get_model_loss(model, positive_sample, negative_sample)

    # apply loss
    loss.backward()
    optimizer.step()

    log = {
        'positive_sample_loss': positive_sample_loss.item(),
        'negative_sample_loss': negative_sample_loss.item(),
        'loss': loss.item()
    }

    return log

# Testing
In this section, we will first write a function to get the ranking of a positive entity compared to its corrupted counterparts, and then use that to define a single test step for the model.

#### Task 8

Define a function `get_ranking` which takes in entity scores and the index of the positive entity. The function should return an integer representing the rank of the score of the positive entity in relation to the other entities. Note that a rank of 1 represents having the *lowest* score.

`entity_scores` has size `[nentity]`. Recall from the function we defined further above that we filtered out known true triples by replacing the corrupted heads with the actual head, so some of the entity scores may actually be that of the positive entity, even when they are not in the index of that entity.

*Hint: torch.argsort will be very useful for this task*.

In [30]:
### BEGIN SOLUTION
# One can choose to 'break ties' in this ranking function by consider offsetting the score of positive entity by some small epsilon amount; but I didn't do this below.
def get_ranking(entity_scores: torch.tensor, positive_entity: int) -> int:
    sorted_indices = torch.argsort(entity_scores)
    ranking = torch.where(sorted_indices == positive_entity)[0].item() + 1
    return ranking
### END SOLUTION

We can now define a single test step for the model, using the ranking function to compute MRR, MR, and HITS@k metrics.

In [31]:
def test_step(model, test_triples, all_true_triples, args):
    '''
    Evaluate the model on test or valid datasets
    '''

    model.eval()

    #Prepare dataloader for evaluation
    test_dataloader_head = DataLoader(
        TestDataset(
            test_triples,
            all_true_triples,
            args.nentity,
            args.nrelation,
            'head-batch'
        ),
        batch_size=args.test_batch_size,
        # num_workers=max(1, args.cpu_num//2),
        num_workers=0,
        collate_fn=TestDataset.collate_fn
    )

    test_dataloader_tail = DataLoader(
        TestDataset(
            test_triples,
            all_true_triples,
            args.nentity,
            args.nrelation,
            'tail-batch'
        ),
        batch_size=args.test_batch_size,
        # num_workers=max(1, args.cpu_num//2),
        num_workers=0,
        collate_fn=TestDataset.collate_fn
    )

    test_dataset_list = [test_dataloader_head, test_dataloader_tail]

    logs = []

    step = 0
    total_steps = sum([len(dataset) for dataset in test_dataset_list])

    # torch.no_grad() since we don't need to track gradients when we're testing
    with torch.no_grad():
        for test_dataset in test_dataset_list: # each of head / tail
            for positive_sample, negative_sample, mode in test_dataset:
                # take a sample from the test dataset
                if args.cuda:
                    positive_sample = positive_sample.cuda()
                    negative_sample = negative_sample.cuda()

                batch_size = positive_sample.size(0)
                assert batch_size == 1, 'evaluation batch size must be set to 1'

                # build the negative sample from the entities
                # currently, negative_sample is just a list of entities
                # [1, 14, 3] for Nations
                built_negative_sample = torch.zeros((batch_size, negative_sample.size()[1], 3), dtype=int)
                if args.cuda:
                    built_negative_sample = built_negative_sample.cuda()

                if mode == 'head-batch':
                    built_negative_sample[:, :, 0] = negative_sample
                    built_negative_sample[:, :, 1:3] = positive_sample[:, 1:3].unsqueeze(dim=1).expand((-1, built_negative_sample.size(1), -1))
                else:
                    built_negative_sample[:, :, 2] = negative_sample
                    built_negative_sample[:, :, 0:2] = positive_sample[:, 0:2].unsqueeze(dim=1).expand((-1, built_negative_sample.size(1), -1))

                # get the scores for each entity
                negative_sample = built_negative_sample.reshape((-1, 3))
                entity_scores = model(negative_sample)

                # retrieve the positive entity
                if mode == 'head-batch':
                    positive_entity = positive_sample[:, 0].item()
                elif mode == 'tail-batch':
                    positive_entity = positive_sample[:, 2].item()
                else:
                    raise ValueError('mode %s not supported' % mode)

                # get the ranking of the positive entity
                ranking = get_ranking(entity_scores, positive_entity)

                # compute and append logs
                logs.append({
                    'MRR': 1.0/ranking,
                    'MR': float(ranking),
                    'HITS@1': 1.0 if ranking <= 1 else 0.0,
                    'HITS@3': 1.0 if ranking <= 3 else 0.0,
                    'HITS@10': 1.0 if ranking <= 10 else 0.0,
                })

                if step % args.test_log_steps == 0:
                    print('Evaluating the model... (%d/%d)' % (step, total_steps))

                step += 1

    metrics = {}
    for metric in logs[0].keys():
        metrics[metric] = sum([log[metric] for log in logs])/len(logs)

    return metrics

# Running
To make the running of experiments easier, we will define several help functions.

## Arguments
`argparse` is a very useful library for managing program arguments, particulary when executing from the command line. Default arguments are defined here. If you want to change argument values, do not do it in this block of code, rather change them in the arguments that are passed through to the program (see further below).

In [32]:
def parse_args(args=None):
    parser = argparse.ArgumentParser(
        description='Training and Testing Knowledge Graph Embedding Models',
        usage='train.py [<args>] [-h | --help]'
    )

    parser.add_argument('--cuda', action='store_true', help='use GPU')

    parser.add_argument('--do_train', action='store_true')
    parser.add_argument('--do_valid', action='store_true')
    parser.add_argument('--do_test', action='store_true')
    parser.add_argument('--evaluate_train', action='store_true', help='Evaluate on training data')

    parser.add_argument('--model', default='TransE', type=str)
    parser.add_argument('-de', '--double_entity_embedding', action='store_true')
    parser.add_argument('-dr', '--double_relation_embedding', action='store_true')

    parser.add_argument('-n', '--negative_sample_size', default=128, type=int)
    parser.add_argument('-d', '--hidden_dim', default=100, type=int, help='Embedding size')
    parser.add_argument('-g', '--gamma', default=2.0, type=float, help='Fixed margin parameter')
    parser.add_argument('-b', '--batch_size', default=1024, type=int)
    parser.add_argument('--test_batch_size', default=1, type=int, help='valid/test batch size (must be 1)')

    parser.add_argument('-lr', '--learning_rate', default=0.0001, type=float)
    parser.add_argument('-cpu', '--cpu_num', default=1, type=int)
    parser.add_argument('--max_steps', default=100, type=int)

    parser.add_argument('--valid_steps', default=20, type=int, help='how often to check accuracy on validation dataset')
    parser.add_argument('--log_steps', default=10, type=int, help='train log every xx steps')
    parser.add_argument('--test_log_steps', default=1000, type=int, help='valid/test log every xx steps')

    parser.add_argument('--nentity', type=int, default=0, help='DO NOT MANUALLY SET')
    parser.add_argument('--nrelation', type=int, default=0, help='DO NOT MANUALLY SET')

    parser.parse_args(args)
    return parser.parse_args(args)

## Logging
The below function is used to help with logging metrics.

In [33]:
def log_metrics(mode, step, metrics):
    '''
    Print the evaluation logs
    '''
    for metric in metrics:
        print('%s %s at step %d: %f' % (mode, metric, step, metrics[metric]))

## Main Program Loop
We can finally bring everything we've done together into the main program loop. Please refer to the comments in the code to understand how it operates.

In [34]:
def main(args):
    if (not args.do_train) and (not args.do_valid) and (not args.do_test):
        raise ValueError('one of train/val/test mode must be chosen.')

    # use CUDA if possible
    args.cuda = torch.cuda.is_available()

    # print dataset parameters
    entity2id = dataset.training.entity_to_id
    relation2id = dataset.training.relation_to_id

    nentity = len(entity2id)
    nrelation = len(relation2id)

    args.nentity = nentity
    args.nrelation = nrelation

    print('Model: %s' % args.model)
    print('#entity: %d' % nentity)
    print('#relation: %d' % nrelation)

    print('#train: %d' % len(train_triples))
    print('#valid: %d' % len(valid_triples))
    print('#test: %d' % len(test_triples))

    # all true triples
    all_true_triples = train_triples + valid_triples + test_triples

    # create the model
    kge_model = KGEModel(
        model_name=args.model,
        nentity=nentity,
        nrelation=nrelation,
        hidden_dim=args.hidden_dim,
        gamma=args.gamma,
        double_entity_embedding=args.double_entity_embedding,
        double_relation_embedding=args.double_relation_embedding
    )

    # output the model params
    print('\nModel Parameter Configuration:')
    for name, param in kge_model.named_parameters():
        print('Parameter %s: %s, require_grad = %s' % (name, str(param.size()), str(param.requires_grad)))

    if args.cuda:
        kge_model = kge_model.cuda()

    if args.do_train:
        # set training dataloader iterator
        train_dataloader = DataLoader(
            TrainDataset(train_triples, nentity, nrelation, args.negative_sample_size),
            batch_size=args.batch_size,
            shuffle=True,
            # num_workers=max(1, args.cpu_num//2),
            num_workers=0,
            collate_fn=TrainDataset.collate_fn
        )
        train_iterator = OneShotIterator(train_dataloader)

        # set training configuration
        optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, kge_model.parameters()),
            lr=args.learning_rate
        )

    init_step = 1
    step = init_step

    # do an initial evaluation to see metrics for a random model
    print('\nEvaluating initial model on Valid Dataset...')
    metrics = test_step(kge_model, valid_triples, all_true_triples, args)
    log_metrics('Valid', step, metrics)

    # log training parameters
    print('\nStart Training...')
    print('init_step = %d' % init_step)
    print('batch_size = %d' % args.batch_size)
    print('hidden_dim = %d' % args.hidden_dim)
    print('gamma = %f' % args.gamma)

    if args.do_train:
        print('learning_rate = %f' % args.learning_rate)

        training_logs = []

        # training loop
        for step in range(init_step, args.max_steps):
            # perform a single train step
            log = train_step(kge_model, optimizer, train_iterator, args)

            # record the logs
            training_logs.append(log)

            # check if logs should be displayed
            if step % args.log_steps == 0:
                metrics = {}
                for metric in training_logs[0].keys():
                    metrics[metric] = sum([log[metric] for log in training_logs])/len(training_logs)
                print('\nTraining metrics...')
                log_metrics('Training average', step, metrics)
                training_logs = []

            # check if metrics should be reported on the validation set
            if args.do_valid and step % args.valid_steps == 0:
                print('\nEvaluating on Valid Dataset...')
                metrics = test_step(kge_model, valid_triples, all_true_triples, args)
                log_metrics('Valid', step, metrics)

    # compute final metrics after training is complete

    if args.do_valid:
        print('Evaluating on Valid Dataset...')
        metrics = test_step(kge_model, valid_triples, all_true_triples, args)
        log_metrics('Valid', step, metrics)

    if args.do_test:
        print('Evaluating on Test Dataset...')
        metrics = test_step(kge_model, test_triples, all_true_triples, args)
        log_metrics('Test', step, metrics)

    if args.evaluate_train:
        print('Evaluating on Training Dataset...')
        metrics = test_step(kge_model, train_triples, all_true_triples, args)
        log_metrics('Test', step, metrics)

The below code can be used to run the main program loop. Model arguments can be adjusted by changing / adding / removing the arguments.

In [35]:
if __name__ == '__main__':
    main(parse_args(['--do_train', '--do_valid', '--do_test',
                     '--model', 'TransE',
                     '--max_steps', '1000', '--valid_steps', '20', '--log_steps', '10']))

Model: TransE
#entity: 14
#relation: 55
#train: 1592
#valid: 199
#test: 201

Model Parameter Configuration:
Parameter gamma: torch.Size([1]), require_grad = False
Parameter embedding_range: torch.Size([1]), require_grad = False
Parameter entity_embedding: torch.Size([14, 100]), require_grad = True
Parameter relation_embedding: torch.Size([55, 100]), require_grad = True

Evaluating initial model on Valid Dataset...
Evaluating the model... (0/398)
Valid MRR at step 1: 0.149247
Valid MR at step 1: 8.258794
Valid HITS@1 at step 1: 0.000000
Valid HITS@3 at step 1: 0.075377
Valid HITS@10 at step 1: 0.743719

Start Training...
init_step = 1
batch_size = 1024
hidden_dim = 100
gamma = 2.000000
learning_rate = 0.000100

Training metrics...
Training average positive_sample_loss at step 10: 1.468488
Training average negative_sample_loss at step 10: 0.329678
Training average loss at step 10: 1.798167

Training metrics...
Training average positive_sample_loss at step 20: 1.378018
Training average ne

In [36]:
if __name__ == '__main__':
    main(parse_args(['--do_train', '--do_valid', '--do_test',
                     '--model', 'RotatE',
                     '--max_steps', '1000', '--valid_steps', '20', '--log_steps', '10',
                     '--double_entity_embedding']))

Model: RotatE
#entity: 14
#relation: 55
#train: 1592
#valid: 199
#test: 201

Model Parameter Configuration:
Parameter gamma: torch.Size([1]), require_grad = False
Parameter embedding_range: torch.Size([1]), require_grad = False
Parameter entity_embedding: torch.Size([14, 200]), require_grad = True
Parameter relation_embedding: torch.Size([55, 100]), require_grad = True

Evaluating initial model on Valid Dataset...
Evaluating the model... (0/398)
Valid MRR at step 1: 0.149310
Valid MR at step 1: 8.268844
Valid HITS@1 at step 1: 0.000000
Valid HITS@3 at step 1: 0.067839
Valid HITS@10 at step 1: 0.743719

Start Training...
init_step = 1
batch_size = 1024
hidden_dim = 100
gamma = 2.000000
learning_rate = 0.000100

Training metrics...
Training average positive_sample_loss at step 10: 0.196188
Training average negative_sample_loss at step 10: 1.781521
Training average loss at step 10: 1.977709

Training metrics...
Training average positive_sample_loss at step 20: 0.199367
Training average ne

One may notice that both the Valid and Test HITS@1 are 0 for the RotatE model. I do think this is just the limitation of this implementation rather than flaws in the coding.